In [17]:
from copy import deepcopy
from string import punctuation
from random import shuffle
import numpy as np


from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.tokenize import TweetTokenizer
import gc

from keras.models import Sequential
from keras.layers import Activation,Embedding,MaxPooling1D,LSTM,Bidirectional
from keras.callbacks import EarlyStopping 
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam
from keras.layers.core import Dense, Dropout
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
#Dictionary size.top Max_word_count words
Max_word_count=300000


In [20]:
#use only smaller set for memory problems
neg_twit_path='data/train_neg.txt'
pos_twit_path='data/train_pos.txt'
test_tweet_path='data/test_data.txt'
test_tweet_only_path='data/test_tweet_data.txt'
neg_twit_cleaned='cleaned twits/train_neg.txt'
pos_twit_cleaned='cleaned twits/train_pos.txt'
test_tweet_cleaned='cleaned twits/test_data.txt'
gloveFile='Glove embeddings/glove.twitter.27B.50d.txt'



In [3]:
def cleanTweets(tweets):
    
    remove_numbers_start = r'\d+(\w*|\d*)'
    remove_numbers_end = r'(\w*|\d*)\d+'
    remove_hashtags = r'#(\w*|#*|\d*)'
    remove_underscores = r'_(\w*)'
    remove_punctuation = r"[.,;'?():-_!$&%{}~/|]"
    remove_quotation = r'"'
    remove_math_ops = r'[-+.^:,*]'  
    remove_delimeted_words = r'<[^>]+>'
    
    combined_pattern = r'|'.join((remove_numbers_start,remove_numbers_end,remove_hashtags,remove_underscores,remove_quotation,remove_delimeted_words))
    combined_pattern2 = r'|'.join((remove_punctuation,remove_math_ops))
    cleaned_tweets1 = re.sub(combined_pattern,'',tweets)
    cleaned_tweets2 = re.sub(combined_pattern2,'',cleaned_tweets1)
    print "clean tweets: ",len(cleaned_tweets2)
    return cleaned_tweets2

In [4]:
# replaces all words having an instance of a character more than 2 timess
def repl(matchObj):	
    char = matchObj.group(1)
    return "%s%s" % (char, char)

In [5]:
def replaceRepeatingCharacters(tweets):
    pattern = re.compile(r"(\w)\1+")
    corrected_words = pattern.sub(repl,tweets)
    return corrected_words

In [6]:
def loadData(path,inputF, outputTweets):
    print 'Loading raw data, cleaning and writing cleaned data...'

    inputSet = path + inputF
    outputSetA = outputTweets
    
    #read data sets
    file = open(inputSet)
    raw_data = file.read()
    print "raw_data:",len(raw_data)

    
    cleaned_data = cleanTweets(raw_data)

    strip_repeated_chars = replaceRepeatingCharacters(cleaned_data)
    
    
    writeToFile(strip_repeated_chars,outputSetA)


In [8]:
def writeToFile(data,outputSet):
    print "writing to file..."
    result = open(outputSet, 'w')
    if (type(data) is list):
        for item in data:
            result.write("%s\n" % item)
    elif (type(data) is str):
        # remove whitespaces generated by data cleaning
        newform = re.sub(' +',' ',data.lstrip())
        result.write(newform)
        


In [9]:
#load glove model
def create_glove_model():
    print "Loading Glove Model..."
    f = open(gloveFile,'r')
    glove_model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove_model[word] = embedding
    print "Done.",len(glove_model)," words loaded!"
    return glove_model

In [10]:
#create tokenizer,create dictionary with sequences with same size
#and Y values like [1.0 0.0] and [0.0 1.0]
def tokenize(corpus,Y,max_tweet_lenght):
    tknzr = Tokenizer(num_words=Max_word_count)
    tknzr.fit_on_texts(corpus)
    sqncs = tknzr.texts_to_sequences(corpus)
    word_index = tknzr.word_index
    X_processed = pad_sequences(sqncs, maxlen=max_tweet_lenght)
    Y_processed = to_categorical(np.asarray(Y), 2)

    return X_processed, Y_processed,word_index

In [11]:
#load cleaned and written clean tweets before
def load_clean_twits(filename):
    print 'File reading and loading clean tweets...'
    data = open(filename, 'rb')
    clean_tweet_all = []
    for tweet in tqdm(data):
        clean_tweet_all.append(tweet)
    return np.array(clean_tweet_all)

In [12]:
#merge positive and negative dataset
def merge_dataset(pos_data, neg_data):
    print 'Merging positive and negative dataset...'
    X = np.concatenate((pos_data, neg_data))
    y = np.array([1] * pos_data.shape[0] + [0] * neg_data.shape[0])
    assert len(y) == X.shape[0]
    assert X.shape[0] == pos_data.shape[0] + neg_data.shape[0]
    return X, y

In [13]:
#create embeddings for word index items 
def define_embedding(glove_model,word_index,n_dim,max_tweet_length):
    embeddings = np.zeros((len(word_index)+1, n_dim))
    print len(word_index)
    for word, i in word_index.items():
        try:
            embedding_vec = glove_model[word]
            embeddings[i] = embedding_vec

        except KeyError:
                continue
                    
    return embeddings
        

In [14]:
#tweakable parameters
max_tweet_length = 80
n_dim=50
batch_size = 64
nb_epochs = 15

loadData('../',neg_twit_path,neg_twit_cleaned)
loadData('../',pos_twit_path,pos_twit_cleaned)

pos_clean_twits=load_clean_twits(pos_twit_cleaned)
neg_clean_twits=load_clean_twits(neg_twit_cleaned)
X, y = merge_dataset(pos_clean_twits,neg_clean_twits)

glove_model=create_glove_model()

X_p, Y_p,word_index=tokenize(X,y,max_tweet_length)

X_train, X_test, Y_train, Y_test = train_test_split(X_p, Y_p, test_size=0.2, random_state=42)

embeddings=define_embedding(glove_model,word_index,n_dim,max_tweet_length)

#X_trim=trim_dataset(X,max_tweet_length,tfidf,glove_model)

#calculate_max_avarage_lenght(X_trim)





#print X_train.shape



Loading raw data, cleaning and writing cleaned data...
raw_data: 8507760
clean tweets:  7231862
writing to file...
Loading raw data, cleaning and writing cleaned data...
raw_data: 6871220
clean tweets:  5691794
writing to file...


95467it [00:00, 2860963.71it/s]
97354it [00:00, 2564434.06it/s]
0it [00:00, ?it/s]

File reading and loading clean tweets...
File reading and loading clean tweets...
Merging positive and negative dataset...
Loading Glove Model...


1193517it [00:15, 78710.98it/s]


Done. 1193515  words loaded!
82295


In [18]:
model = Sequential()
#trainable is false because we use frozen glove vectors that already trained
model.add(Embedding(len(word_index)+1, n_dim, weights=[embeddings], input_length=max_tweet_length, trainable=False))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
    # Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])


In [19]:

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=nb_epochs,
          validation_data=(X_test, Y_test))

score, acc = model.evaluate(X_test, Y_test,batch_size=batch_size)

Train on 154256 samples, validate on 38565 samples
Epoch 1/10
154256/154256 [==============================] - 469s 3ms/step - loss: 0.4635 - acc: 0.7680 - val_loss: 0.4282 - val_acc: 0.7917
Epoch 2/10
154256/154256 [==============================] - 467s 3ms/step - loss: 0.4217 - acc: 0.7961 - val_loss: 0.4078 - val_acc: 0.8063
Epoch 3/10
154256/154256 [==============================] - 467s 3ms/step - loss: 0.4091 - acc: 0.8055 - val_loss: 0.4108 - val_acc: 0.8051
Epoch 4/10
154256/154256 [==============================] - 471s 3ms/step - loss: 0.4018 - acc: 0.8093 - val_loss: 0.3958 - val_acc: 0.8135
Epoch 5/10
154256/154256 [==============================] - 485s 3ms/step - loss: 0.3959 - acc: 0.8130 - val_loss: 0.3988 - val_acc: 0.8105
Epoch 6/10
154256/154256 [==============================] - 476s 3ms/step - loss: 0.3910 - acc: 0.8156 - val_loss: 0.3936 - val_acc: 0.8143
Epoch 7/10
154256/154256 [==============================] - 484s 3ms/step - loss: 0.3856 - acc: 0.8184 - val_